In [58]:
!pip install langchain
!pip install langchain_openai
!pip install langgraph
!pip install langchain_community
!pip install mesop

4078.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


4085.95s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


4093.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


4100.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


4107.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 52.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 7.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
 
 
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

from langchain_openai import AzureChatOpenAI
import func
import os

os.environ["AZURE_OPENAI_API_KEY"] = 'b3e819600fbe4981be34ef2aa79943e2'



f = func.Func()

def main():
  tools = [f.getingPolicyInfo, f.getCustomerID, f.sendEmail, f.getUserInfo]
  username = "Olivia"

  llm = AzureChatOpenAI(
      azure_endpoint="https://ik-oai-eastus-2.openai.azure.com/",
      azure_deployment="gpt-4o",
      openai_api_version="2023-09-01-preview",
  )

  agent_executor = create_react_agent(llm, tools)
  prompt = {"messages": [HumanMessage(content=f"extract the policy information, get user information of {username}, phone number is 909324 and send it to asa.choi@gmail.com")]}

  for chunk in agent_executor.stream(prompt):
      print(chunk)
      if(chunk.get("tools")):
        m = chunk["tools"]["messages"][0]
        print(f"{m.name}, {m.content}")

#sendEmail("asa.choi@gmail.com", "how are you?")
main();

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GAcYDIk9d3f6nna4jWHmboKa', 'function': {'arguments': '{"name":"Olivia","phone":"909324"}', 'name': 'getCustomerID'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 180, 'total_tokens': 201}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-8fd3aaed-5f52-49ae-8c26-89d85250df4a-0', tool_calls=[{'name': 'getCustomerID', 'args': {'name': 'Olivia', 'phone': '909324'}, 'id': 'call_GAcYDIk9d3f6nna4jWHmboKa'}], usage_metadata={'input_tokens': 180, 'output_tokens': 21, 'total_tokens': 201})]}}
{'tools': {'messages': [ToolMessage(content='12345', name='getCustomerID', tool_call_id='call_GAcYDIk9d3f6nna4jWHmboKa')]}}
getCustomerID, 12345
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GAcYDIk9d3f6n